<a href="https://colab.research.google.com/github/ITA-LOW/MTM3587-08222-2021-2-Aprendizado-de-Maquina/blob/main/Filtro_de_SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Mini projeto Naive Bayes e filtro de SPAM


## Objetivos do projeto

*   Responder/implementar corretamente todos os tópicos em "ToDo";
*   Explicar o que significa o "Bag of Words";
* Explicar a diferença entre especificidade e sensitividade. Dê dois exemplos práticos, uma em que a especificidade parece ser mais adequada do que a sensitividade e vice-versa;
* Implementar o Naive Bayes conforme indica o roteiro e comparar com um (1) dos algoritmos vistos no ML tour, justificando a escolha do melhor modelo. É para comparar o Naive Bayes com um e apenas um algoritmo;
* Você deverá fazer uma lista dos prós e contras do Naive Bayes e do algoritmo escolhido. Você deve explicar o resultado obtido com base nas características dos dois algoritmos avaliado. Você teria algum insight relevante para me apresentar?



## Roteiro

1. Conceito de "Bag of Words".
2. Conceito de especificidade e sensibilidade.
1. Análise exploratória do conjunto de dados.
1. Exemplo de aplicação do BoW.
3. Implementando o algoritmo Naive Bayes ao conjunto de dados.
1. Apresentação do Teorema de Naive Bayes.
1. Aplicando o Naive Bayes ao projeto Filtro de SPAM.
1. Avaliando o modelo.
1. Comparando o Naive Bayes com a Árvore de decisão.
1. Conclusão.
1. Referência.






# 1. Conceito de "Bag of Words"
A maioria dos algoritmos de aprendizado de máquina usa números como dados de entrada e mensagens de e-mail ou sms são basicamente dados de texto. A ideia do "Bag of Words" é separar cada palavra do conjunto de dados e contar a frequência com que elas aparecem retornando, assim, um número para nosso algoritmo.
O processo para implementar o BoW é converter o conjunto de dados em uma matriz, onde cada palavra (token) é uma coluna e cada mensagem é uma linha. Para realizar essa implementação será usada o método count vectorizer da biblioteca sklearn.

## 2. Conceito de Especificidade e Sensibilidade

* <h3> 2.1 Especificidade<h3>

Especificidade é uma métrica que nos mostra a capacidade de um algoritmo de detectar corretamente um verdadeiro negativo. No caso do nosso conjunto de dados, ele mede a capacidade do algoritmo de detectar corretamente se a mensagem recebidade é "ham". A especificidade pode ser obtida fazendo a razão: $$ \frac {VN}{VN+FP}\ $$ onde:

VN = Verdadeiro negativo (a mensagem é "ham")

FP = Falso positivo (o classificador errou por identificar a mensagem como "ham")



* <h3> 2.2 Sensibilidade<h3>

Sensibilidade é uma métrica que avalia a capacidade do algoritmo de detectar com sucesso resultados classificados como verdadeiro positivo. No caso do nosso conjunto de dados, mede a capacidade do algoritmo de detectar se uma mensagem foi corretamente classificada como SPAM. A sensibilidade pode ser obtida atravéz da razão: $$ \frac{VP}{VP+FN}\ $$ onde:

VP = Verdadeiro positivo (a mensagem é SPAM)

FN = Falso negativo (o classificador errou por identificar uma mensagem como SPAM)



No contexto deste projeto, a métrica "sensibilidade" parece ser mais adequada já que o que se espera desse algoritmo é uma boa relação entre a quantidade de mensagens corretamente classificadas como SPAM em face ao conjunto das mensagens que de fato são SPAM e que foram erroneamente classificadas como (VP+FN).

Por outro lado, um caso onde é mais vantajoso usar a métrica "especificidade" para medir a eficiência do algoritmo seria quando se quer identificar situações onde a maioria dos dados é verdadeira negativa.

# 3. Análise exploratória do conjunto de dados
Vamos iniciar trazendo o dataset a ser trabalhado obtido do repositório https://github.com/udacity/machine-learning/blob/master/projects/practice_projects/naive_bayes_tutorial/Bayesian_Inference.ipynb para o dataframe.

In [ ]:
import pandas as pd
df=pd.read_table('/content/SMSSpamCollection', sep="\t", header=None, names=['Tipo', 'Conteudo'])

df.head()

,Tipo,Conteudo
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Podemos notar que o "Tipo" da mensagem é literal, como podemos classificar em binário (SPAM/não-SPAM) vamos tratar esse dado do dataset como 0-"ham" e 1-SPAM.

In [ ]:
df['Tipo']=df.Tipo.map({'ham':0,'spam':1})

df.head()

,Tipo,Conteudo
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Para saber como é este dataset utilizamos o parâmetro shape do método df.

In [ ]:
print(df.shape)

(5572, 2)


Este dataset tem 5572 mensagens e apenas as colunas "Tipo" e "Conteúdo".

A próxima etapa é a de implementação do BoW. O BoW consiste em uma fase de limpeza e preparação dos dados utilizando o método CountVectorizer do Sklearn porém antes de continuar a análise exploratória dos dados faremos uma apresentação didática de como esse método funciona no exemplo abaixo.

# 4. Exemplo da aplicação do BoW:
Vamos usar o seguinte dataset como exemplo:

    documento = ['Olá, como vai!',
                'Ganhe dinheiro perto de Florianópolis!',
                'Me ligue agora, urgente!',
                'Olá, posso te ligar amanhã?']



###Convertendo as palavras para minúsculas!

In [ ]:
documento=['Olá, como vai!',
            'Ganhe dinheiro perto de Florianópolis!',
            'Me ligue agora, urgente!',
            'Olá, posso te ligar amanhã?']
documento_caixa_baixa=[]
for i in documento:
  documento_caixa_baixa.append(i.lower())
print(documento_caixa_baixa)



['olá, como vai!', 'ganhe dinheiro perto de florianópolis!', 'me ligue agora, urgente!', 'olá, posso te ligar amanhã?']


###Removendo a pontuação da lista "documento_caixa_baixa"!

In [ ]:
doc_sem_ponto=[]
import string
for i in documento_caixa_baixa:
  doc_sem_ponto.append(i.translate(str.maketrans(' ',' ',string.punctuation)))
print(doc_sem_ponto)

['olá como vai', 'ganhe dinheiro perto de florianópolis', 'me ligue agora urgente', 'olá posso te ligar amanhã']


###Realizando a "tokenização"
Tokenizar consiste em dividir a mensagem em palavras atravéz de um delimitador (nesse caso o "espaço") o qual delimitará o que é o início e o fim de uma palavra.

In [ ]:
doc_token=[]
for i in doc_sem_ponto:
  doc_token.append(i.split(' '))
print(doc_token)

[['olá', 'como', 'vai'], ['ganhe', 'dinheiro', 'perto', 'de', 'florianópolis'], ['me', 'ligue', 'agora', 'urgente'], ['olá', 'posso', 'te', 'ligar', 'amanhã']]


###Contando palavras
Nesta etapa, será implementado o método Counter() que conta a frequencia que ocorre cada palavra dentro da mensagem.

In [ ]:
frequencia=[]
import pprint
from collections import Counter

for i in doc_token:
  frequencia_ocorrida=Counter(i)
  frequencia.append(frequencia_ocorrida)
pprint.pprint(frequencia)


[Counter({'olá': 1, 'como': 1, 'vai': 1}),
 Counter({'ganhe': 1, 'dinheiro': 1, 'perto': 1, 'de': 1, 'florianópolis': 1}),
 Counter({'me': 1, 'ligue': 1, 'agora': 1, 'urgente': 1}),
 Counter({'olá': 1, 'posso': 1, 'te': 1, 'ligar': 1, 'amanhã': 1})]


Com isso o conceito de BoW foi corretamente aplicado

###Implementando o BoW com Sklearn
Agora, tendo assimilado o que ocorre ao fazer a limpeza e tratamento do dataset utilizando o conceito de BoW, farei a implementação desse método nativo da biblioteca Sklearn no mesmo documento inicial.

In [ ]:
#importando o método criando a instância vetor_contador
from sklearn.feature_extraction.text import CountVectorizer
vetor_contador = CountVectorizer()

In [ ]:
#Ajustando o dataset "documento" ao método CountVectorizer()
vetor_contador.fit(documento)

#Obtendo uma lista do que será usado como característica (colunas)
vetor_contador.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['agora',
 'amanhã',
 'como',
 'de',
 'dinheiro',
 'florianópolis',
 'ganhe',
 'ligar',
 'ligue',
 'me',
 'olá',
 'perto',
 'posso',
 'te',
 'urgente',
 'vai']

In [ ]:
#Converto o dataset numa matriz onde cada linha é uma mensagem (já tratada) e cada coluna é uma característica (vista ao passar o método .get_feature_names())
matriz_doc=vetor_contador.transform(documento).toarray()
matriz_doc

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]])

In [ ]:
#Finalmente, para melhor visualização, será criado um dataframe unindo essa matriz obtida com as características 
matriz_final = pd.DataFrame(matriz_doc, columns=vetor_contador.get_feature_names())

matriz_final

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,agora,amanhã,como,de,dinheiro,florianópolis,ganhe,ligar,ligue,me,olá,perto,posso,te,urgente,vai
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
3,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0


Com isso, finalizamos a apresentação de como aplicar o BoW num dataset. Voltamos ao problema: preparar um filtro anti-SPAM ao conjunto de dados adquirido inicialmente.

#5. Implementando o algoritmo Naive Bayes ao conjunto de dados

In [ ]:
#Dividindo o conjunto em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Conteudo'], 
                                                    df['Tipo'], 
                                                    random_state=1)
print('Total de linhas em todo o conjunto: {}'.format(df.shape[0]))
print('Total de linhas no conjunto de treino: {}'.format(X_train.shape[0]))
print('Total de linhas no conjunto de teste: {}'.format(X_test.shape[0]))

Total de linhas em todo o conjunto: 5572
Total de linhas no conjunto de treino: 4179
Total de linhas no conjunto de teste: 1393


###Aplicando o BoW nos conjuntos de treino e teste

In [ ]:
#Instanciando o vetor_contador
vetor_contador=CountVectorizer()

#Ajustando os dados de treino para retornar uma matriz
dados_treino=vetor_contador.fit_transform(X_train) 

#Ajustando os dados de teste para retornar uma matriz
dados_teste=vetor_contador.transform(X_test)


#6. Apresentação do Teorema de Naive Bayes
Agora que temos o dataset no formato que precisamos será introduzido teorema de Bayes. O teorema de Naive Bayes calcula a probabilidade de um evento acontecer baseado na ocorrência de outros eventos relacionados a ele. 

Será usado como exemplo o cálculo da probabilidade de um indivíduo ter Diabetes dado que foi testado positivo.

Os seguintes dados serão assumidos:

**P(D)** é a probabilidade de uma pessoa ter Diabetes. Assume-se que 1% da população geral tem Diabetes;

**P(POS)** é a probabilidade de um resultado positivo no teste;

**P(NEG)** é a probabilidade de um resultado negativo no teste;

**P(POS|D)** é a probabilidade de dar positivo no teste e o indíviduo realmente ter Diabetes(verdadeiro positivo). Esse valor também é chamado de *sensibilidade*. O teste acerta 90% das vezes.

**P(NEG|~D)** é a probabilidade de dar negativo no teste e o indivíduo não ter Diabetes. Esse valor também é chamado de *especificidade*. Neste caso, o teste também acerta 90% das vezes.

A fórmula de Bayes é a seguinte: $$ P(A|B)= \frac {P(B|A) P(A)}{P(B)}\ $$ onde:

$P(A|B)$ é a probabilidade de ocorrer $A$ dado $B$. No exemplo, esse é o valor de o indivíduo ter Diabete dado que o resultado do seu teste foi positivo. Esse é o valor buscado.

$P(B|A)$ é a probabilidade de ocorrer $B$ dado $A$. No exemplo, esse é o valor da *sensibilidade*, ou seja, o teste do indíviduo ter dado positivo e ele realmente ter Diabetes. Esse valor é 90%.

$P(A)$ é a probabilidade de ocorrer o evento $A$, neste exemplo, é a probabilidade de o indivíduo ter Diabetes independente de qualquer outro dado. Assumimos que a população no geral tem 1% de chance de ter Diabetes.

$P(B)$ é a probabilidade de ocorrer o evento $B$, neste exemplo, é a probabilidade do teste de um indíviduo dar positivo independente de qualquer outro dado.

Substituindo na fórmula de Naive Bayes:

$$P(D|POS)= \frac {P(POS|D) P(D)}{P(POS)}\ $$

A probabilidade do teste dar positivo ($P(POS)$) pode ser calculada usando a especificidade e sensibilidade: $$P(POS)=[P(D)\times(sensibilidade)+P(nãoD)\times(1-especificidade)]\ $$




In [ ]:
#P(D)
p_d=0.01

#P(~D)
p_nd=0.99

#sensibilidade ou P(POS|D)
pos_d=0.9

#especificidade ou P(NEG|~D)
neg_d=0.9

p_pos = (p_d*pos_d) + (p_nd*(1 - neg_d))

print('A probabilidade de ocorrer um resultado positivo em um indivíduo qualquer é {}'.format(p_pos))

A probabilidade de ocorrer um resultado positivo em um indivíduo qualquer é 0.10799999999999998


In [ ]:
#A probabilidade de um indivíduo ter Diabetes dado que obteve um resultado positivo no teste
p_d_pos=(p_d*pos_d)/p_pos
print('A probabilidade de ocorrer um verdadeiro positivo é {}'.format(p_d_pos))

A probabilidade de ocorrer um verdadeiro positivo é 0.08333333333333336


In [ ]:
#A probabilidade de um indivíduo não ter Diabetes dado que seu resultado foi positivo
nd_pos=p_nd*(1-neg_d)/p_pos
print('A probabilidade de ocorrer um falso positivo é {}'.format(nd_pos))

A probabilidade de ocorrer um falso positivo é 0.9166666666666666


Com isso, terminamos a apresentação do Teorema de Naive Bayes

#7. Aplicando o Naive Bayes ao projeto Filtro de SPAM
Já temos os 2 conjuntos de treino e teste alcançados no item 5 deste trabalho. Vamos ajustar esses 2 conjuntos ao classificador MultinomialNB.

In [ ]:
#Chamando o método MultinomialNB
from sklearn.naive_bayes import MultinomialNB
#Instanciando o método como naive_bayes
naive_bayes = MultinomialNB()
naive_bayes.fit(dados_treino, y_train)

MultinomialNB()

In [ ]:
predicao = naive_bayes.predict(dados_teste)

#8. Avaliando o modelo
A avaliação será feita utilizando o conjunto de teste e a predição que foi feita e armazenada na variável "predicao"

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
acuracia = accuracy_score(y_test, predicao)
especificidade = precision_score(y_test, predicao)
sensibilidade = recall_score(y_test, predicao)
f1 = f1_score(y_test, predicao)

print('Acuracia: ',acuracia)
print('Especificidade:',especificidade)
print('Sensibilidade:',sensibilidade)
print('Média harmônica entre Especificidade e Sensibilidade:',f1)

Acuracia:  0.9885139985642498
Especificidade: 0.9720670391061452
Sensibilidade: 0.9405405405405406
Média harmônica entre Especificidade e Sensibilidade: 0.9560439560439562


#9. Comparando o Naive Bayes com a Árvore de decisão.
Farei uma rápida comparação com o algoritmo de árvore de decisão para testar sua performance.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
arvore_decisao=DecisionTreeClassifier()
arvore_decisao
arvore_decisao.fit(dados_treino, y_train)
predicao_tree = arvore_decisao.predict(dados_teste)

#Avaliando o modelo
acuracia_tree = accuracy_score(y_test, predicao_tree)
especificidade_tree = precision_score(y_test, predicao_tree)
sensibilidade_tree = recall_score(y_test, predicao_tree)
f1_tree = f1_score(y_test, predicao_tree)

print(acuracia_tree,
      especificidade_tree,
      sensibilidade_tree,
      f1_tree)

0.9626704953338119 0.844559585492228 0.8810810810810811 0.8624338624338624


Apesar de ser necessário um aprofundamento no algoritmo de árvore de decisão, com essa pequena amostra é possível ver uma diferença relevante.
Em comparação, esses dois algoritmos apresentam prós e contras:
<center>
<h1>Naive Bayes

</center>

* Prós:
 * Fácil implementação
 * Rápido e simples
 * Resistente a ruído: caso de datasets com muitos outliers, muitas características correlatas... O algoritmo consegue ignorar esses ruídos pois trata as features (características) independentemente (por isso o nome Naive)
 * Se ajusta bem a grandes conjuntos de dados

* Contras: 
 * Não performa bem com features que dependem umas das outras
 * Performa estritamente bem em algoritmos de classificação, tendo perda de eficiência em datasets numéricos
 * Comportamento tendencioso: por tratar as características de maneira independente pode ocorrer que uma delas influencie no resultado de maneira irreal.

<center>
<h1>Árvore de decisão

</center>

* Prós:
 * Requer menos etapas de preparação dos dados
 * Valores faltantes não afetam o processo de construção do algoritmo
 * Não é necessário fazer dimensionamento dos dados, tornando fácil sua implementação

* Contras:
 * É instável: poucas mudanças nos dados podem acarretar em grandes diferenças
 * Pode ser muito complexo dependendo do caso
 * É inadequado para prever regressão e valores contínuos (numéricos)



#10. Conclusão
Diante dessas informações acredito que o filtro de SPAM performaria melhor com o Naive Bayes de fato. Obteve uma performance melhor, é mais claro de se entender. Além disso, as características do conjunto de dados se encaixam melhor modo de funcionamento do algoritmo.

#11. Referências
1. https://github.com/udacity/machine-learning/tree/master/projects/practice_projects/naive_bayes_tutorial
1. MTM3587-08222 (20212) - Aprendizado da Máquina
1. https://dhirajkumarblog.medium.com/top-5-advantages-and-disadvantages-of-decision-tree-algorithm-428ebd199d9a
1. https://holypython.com/nbc/naive-bayes-pros-cons/
1. Introduction to machine learning with Python

```
# Isto está formatado como código
```



#Seguindo o roteiro foi possível:
* Responder/implementar corretamente todos os tópicos em "ToDo" ✔
*   Explicar o que significa o "Bag of Words" ✔
* Explicar a diferença entre especificidade e sensitividade. Dê dois exemplos práticos, uma em que a especificidade parece ser mais adequada do que a sensitividade e vice-versa ✔
* Implementar o Naive Bayes conforme indica o roteiro e comparar com um (1) dos algoritmos vistos no ML tour, justificando a escolha do melhor modelo. É para comparar o Naive Bayes com um e apenas um algoritmo ✔
* Você deverá fazer uma lista dos prós e contras do Naive Bayes e do algoritmo escolhido. Você deve explicar o resultado obtido com base nas características dos dois algoritmos avaliado. ✔